In [1]:
# Mounting the drive 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#importing tensorflow 2 
% tensorflow_version 2.x
import tensorflow as tf
# Importing required Libraries 
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline

TensorFlow 2.x selected.


## Load the dataset

In [0]:
# Setting project path
project_path = '/content/drive/My Drive/Deep Learning/R9_SNLP_Project/ANN -Statistical NLP/'

In [0]:
# setting default path as project path
os.chdir(project_path)

In [5]:
# checking the contents in the project_path 
os.listdir()

['blog-authorship-corpus.zip', 'blogtext.csv', 'SNLP_R9_project1.ipynb']

In [0]:
zip_path = project_path + 'blog-authorship-corpus.zip'

In [0]:
# Extracting the zip file 
from zipfile import ZipFile
with ZipFile(zip_path , 'r') as z:
  z.extractall()

In [0]:
# reading blogtext.csv 
df_full = pd.read_csv('blogtext.csv')

In [9]:
df_full.head()

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


In [10]:
#shape of blogtext.csv
df_full = df_full.iloc[:100000 , :]
df_full.shape

(100000, 7)

In [11]:
# columns of blogtext.csv
df_full.columns

Index(['id', 'gender', 'age', 'topic', 'sign', 'date', 'text'], dtype='object')

In [12]:
# checking memory consuptiom by blogtext.csv
print ('memory consumption ',df_full.memory_usage().sum()/1000000 , 'mb' )

memory consumption  5.600128 mb


In [13]:
df_full.head()

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


In [14]:
df_full.isnull().sum() # No Null values

id        0
gender    0
age       0
topic     0
sign      0
date      0
text      0
dtype: int64

In [15]:
# checking unique astrological sign
df_full.sign.unique()

array(['Leo', 'Aquarius', 'Aries', 'Capricorn', 'Gemini', 'Cancer',
       'Sagittarius', 'Scorpio', 'Libra', 'Virgo', 'Taurus', 'Pisces'],
      dtype=object)

In [16]:
# checking age column range
df_full.age.unique()

array([15, 33, 14, 25, 17, 23, 37, 26, 24, 27, 45, 34, 41, 44, 16, 39, 35,
       36, 46, 42, 13, 38, 43, 40, 47, 48])

In [17]:
# checking unique value of topic column
df_full.topic.unique()

array(['Student', 'InvestmentBanking', 'indUnk', 'Non-Profit', 'Banking',
       'Education', 'Engineering', 'Science', 'Communications-Media',
       'BusinessServices', 'Sports-Recreation', 'Arts', 'Internet',
       'Museums-Libraries', 'Accounting', 'Technology', 'Law',
       'Consulting', 'Automotive', 'Religion', 'Fashion', 'Publishing',
       'Marketing', 'LawEnforcement-Security', 'HumanResources',
       'Telecommunications', 'Military', 'Government', 'Transportation',
       'Architecture', 'Advertising', 'Agriculture', 'Biotech',
       'RealEstate', 'Manufacturing', 'Construction', 'Chemicals',
       'Maritime', 'Tourism', 'Environment'], dtype=object)

In [18]:
df_full.text.tail(5)

99995                THE HINDU - 125 YEARS             ...
99996                DILBERT & IIT-ans                 ...
99997                Case Study : How HP won $3 billion...
99998                Championing Chennai               ...
99999                WEEKEND                         It...
Name: text, dtype: object

In [19]:
# downloading nltk stop words
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
len(stop_words)

179

In [0]:
#data preprocessing , cleaning
import re, string

def clean_str(string):
    try:
        string=re.sub(r'^https?:\/\/<>.*[\r\n]*','',string,flags=re.MULTILINE) #Removing Special characters characters
        string=re.sub(r"[^A-Za-z]"," ",string) # Keeping A-Z and a-z removing rest  
        words=string.strip().lower().split() # Converting text to lowercase
        words=[w for w in words if not w in stop_words] #Remove stopwords
        return " ".join(words) # removes space 
    except:
        return ""

In [0]:
# Applying text cleaning function on df
df_full['processed_text'] = df_full['text'].map(clean_str) 

In [23]:
# comparing orignal text with processed text
df_full[['processed_text','text']].head()

,processed_text,text
0,info found pages mb pdf files wait untill team...,"Info has been found (+/- 100 pages,..."
1,team members drewes van der laag urllink mail ...,These are the team members: Drewe...
2,het kader van kernfusie op aarde maak je eigen...,In het kader van kernfusie op aarde...
3,testing testing,testing!!! testing!!!
4,thanks yahoo toolbar capture urls popups means...,Thanks to Yahoo!'s Toolbar I can ...


In [0]:
# Merging the gender , age, topic and sign columns into a new lables column
df_full['labels'] = df_full['gender'] +',' + df_full['age'].astype(str) + ',' + df_full['topic'] + ',' + df_full['sign']

In [25]:
df_full['labels'].head()

0                   male,15,Student,Leo
1                   male,15,Student,Leo
2                   male,15,Student,Leo
3                   male,15,Student,Leo
4    male,33,InvestmentBanking,Aquarius
Name: labels, dtype: object

In [0]:
# creating new df consisting of processed text and lables
df = df_full[['processed_text','labels']]

In [27]:
df.head()

,processed_text,labels
0,info found pages mb pdf files wait untill team...,"male,15,Student,Leo"
1,team members drewes van der laag urllink mail ...,"male,15,Student,Leo"
2,het kader van kernfusie op aarde maak je eigen...,"male,15,Student,Leo"
3,testing testing,"male,15,Student,Leo"
4,thanks yahoo toolbar capture urls popups means...,"male,33,InvestmentBanking,Aquarius"


## 4. Separate features and labels, and split the data into training and testing

In [0]:
X = df['processed_text']
y = df ['labels']

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

## Vectorize the features

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
vectorizer = CountVectorizer(ngram_range=(1, 2),min_df=2) # Creating bag of words.  

In [33]:
vectorizer.fit(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=2,
                ngram_range=(1, 2), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [34]:
X_train_dtm = vectorizer.fit_transform(X_train)
X_test_dtm = vectorizer.transform(X_test)
print(X_train_dtm.shape)
print(X_test_dtm.shape)

(80000, 898184)
(20000, 898184)


In [35]:
# last 50 features
print(vectorizer.get_feature_names()[20:])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Print the term-document matrix

In [36]:
X_test_dtm

<20000x898184 sparse matrix of type '<class 'numpy.int64'>'
	with 2319276 stored elements in Compressed Sparse Row format>

In [0]:
# Creating empty dictionary 
y_ch = y.iloc[:10]
myDict = dict() # creating empty dict 


In [38]:
for i, j in enumerate(y):
  my_list = j.split(',')
  
  for item in my_list:
    #myDict[item] = my_list.count(item)
    if (item in myDict): 
        myDict[item] += 1
    else: 
         myDict[item] = 1 
    
for key, value in myDict.items(): 
      print ("% s : % d"%(key, value))
    



male :  53358
15 :  6532
Student :  22122
Leo :  8230
33 :  2835
InvestmentBanking :  244
Aquarius :  9050
female :  46642
14 :  3540
indUnk :  33097
Aries :  10637
25 :  8660
Capricorn :  8723
17 :  12755
Gemini :  9225
23 :  10757
Non-Profit :  1326
Cancer :  9253
Banking :  354
37 :  863
Sagittarius :  7366
26 :  8059
24 :  11814
Scorpio :  7049
27 :  8007
Education :  5553
45 :  906
Engineering :  2332
Libra :  7250
Science :  1090
34 :  2388
41 :  772
Communications-Media :  2830
BusinessServices :  626
Sports-Recreation :  406
Virgo :  7134
Taurus :  8530
Arts :  5031
Pisces :  7553
44 :  76
16 :  8406
Internet :  2251
Museums-Libraries :  308
Accounting :  528
39 :  568
35 :  4720
Technology :  8484
36 :  3045
Law :  360
46 :  914
Consulting :  905
Automotive :  124
42 :  156
Religion :  1081
13 :  1497
Fashion :  1898
38 :  801
43 :  505
Publishing :  1079
40 :  513
Marketing :  726
LawEnforcement-Security :  368
HumanResources :  209
Telecommunications :  165
Military :  798
G

## Transform the labels

In [0]:
list_class = [] 
for key in myDict.keys(): 
    list_class.append(key) 
list_class_array=np.array(list_class)

In [0]:
# transform to dictionary as Acceptable format of MultiLabelBinarizer
y_train_pass = [set(i.split(',')) for i in y_train]
y_test_pass = [set(i.split(',')) for i in y_test]

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer

In [42]:
mlb = MultiLabelBinarizer()
mlb.fit(y_train_pass)

MultiLabelBinarizer(classes=None, sparse_output=False)

In [43]:
len(y_train_pass)

80000

In [44]:
# retriving the lables 
mlb.classes_

array(['13', '14', '15', '16', '17', '23', '24', '25', '26', '27', '33',
       '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44',
       '45', '46', '47', '48', 'Accounting', 'Advertising', 'Agriculture',
       'Aquarius', 'Architecture', 'Aries', 'Arts', 'Automotive',
       'Banking', 'Biotech', 'BusinessServices', 'Cancer', 'Capricorn',
       'Chemicals', 'Communications-Media', 'Construction', 'Consulting',
       'Education', 'Engineering', 'Environment', 'Fashion', 'Gemini',
       'Government', 'HumanResources', 'Internet', 'InvestmentBanking',
       'Law', 'LawEnforcement-Security', 'Leo', 'Libra', 'Manufacturing',
       'Maritime', 'Marketing', 'Military', 'Museums-Libraries',
       'Non-Profit', 'Pisces', 'Publishing', 'RealEstate', 'Religion',
       'Sagittarius', 'Science', 'Scorpio', 'Sports-Recreation',
       'Student', 'Taurus', 'Technology', 'Telecommunications', 'Tourism',
       'Transportation', 'Virgo', 'female', 'indUnk', 'male'],
      dtype

In [0]:
y_trn_mlb = mlb.transform(y_train_pass)

In [0]:
y_test_mlb =mlb.transform(y_test_pass)

## Choose a classifier

In [0]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

In [0]:
clf = LogisticRegression(solver='lbfgs',max_iter=4500)
clf = OneVsRestClassifier(clf)

##  Fit the classifier, make predictions and get the accuracy

In [0]:
# fitting the classifier 
clf.fit(X_train_dtm, y_trn_mlb) 

In [0]:
# Predicting on test data 
y_pred_class = clf.predict (X_test_dtm)

In [51]:
 #calculate accuracy of class predictions
from sklearn import metrics
# Score CNN model
metrics.accuracy_score(y_test_mlb, y_pred_class)

0.00085

In [52]:
# Classification report  CNN model
print(metrics.classification_report(y_test_mlb, y_pred_class))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       337
           1       0.00      0.00      0.00       708
           2       0.00      0.00      0.00      1303
           3       0.83      0.01      0.02      1651
           4       0.33      0.00      0.00      2512
           5       0.00      0.00      0.00      2129
           6       0.80      0.00      0.00      2367
           7       0.00      0.00      0.00      1758
           8       1.00      0.00      0.00      1625
           9       0.00      0.00      0.00      1619
          10       0.00      0.00      0.00       539
          11       0.00      0.00      0.00       486
          12       0.00      0.00      0.00       959
          13       0.00      0.00      0.00       663
          14       0.00      0.00      0.00       143
          15       0.00      0.00      0.00       167
          16       0.00      0.00      0.00       120
          17       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score

In [54]:

print("F1: " , (f1_score(y_test_mlb, y_pred_class, average='micro')))
print("F1_macro: " , (f1_score(y_test_mlb, y_pred_class, average='macro')))
print("Recall micro: " , recall_score(y_test_mlb, y_pred_class, average='micro'))
print("F1_micro: " , (f1_score(y_test_mlb, y_pred_class, average='micro')))
print("Recall macro: " , recall_score(y_test_mlb, y_pred_class, average='macro'))
print("Average Precision: " ,(average_precision_score(y_test_mlb, y_pred_class, average='micro')))
print("Accuracy:" , (accuracy_score(y_test_mlb, y_pred_class))) 

F1:  0.24716093526832533
F1_macro:  0.016515124387423287
Recall micro:  0.1546625
F1_micro:  0.24716093526832533
Recall macro:  0.015982836586512412
Average Precision:  0.13737343593136525
Accuracy: 0.00085


## Print true label and predicted label for any five examples

In [0]:
y_test_pred_inversed = mlb.inverse_transform(y_pred_class)
y_test_inversed = mlb.inverse_transform(y_test_mlb)
for i in range(15,20):
    print( 'True labels:\t{}\nPredicted labels:\t{}\n\n'.format(
        
        ','.join(y_test_inversed[i]),
        ','.join(y_test_pred_inversed[i])
    ))

True labels:	36,Pisces,Technology,male
Predicted labels:	Pisces,male


True labels:	25,Cancer,Non-Profit,male
Predicted labels:	male


True labels:	35,Aries,Technology,male
Predicted labels:	male


True labels:	25,Gemini,indUnk,male
Predicted labels:	male


True labels:	17,Virgo,indUnk,male
Predicted labels:	17,Virgo,indUnk,male


